In [23]:
import json, random, re, collections, itertools, base64, sys
from copy import deepcopy
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from operator import itemgetter as at
from IPython.core.display import display, HTML
from ipywidgets import interact
display_html = lambda x: display(HTML(str(x)))
from pprint import pprint as pp
ls = lambda path: print("\n".join(map(str, path.iterdir())))

sys.path.append('../src')
mturk_path = Path("../mturk/results")
data_dir = Path("../data/scone")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
parsed = []
with (data_dir / "alchemy-dev.tsv").open('r') as f:
    for line in f:
        idx, *parts = line.strip().split('\t')
        states       = [{int(v.split(':',1)[0]):v.split(':')[1].strip() for v in p.split(' ')} for i,p in enumerate(parts) if i%2==0]
        instructions = [p for i,p in enumerate(parts) if i%2==1]
        instructions.insert(0,"INIT")
        seq = 0
        for state, instruction in zip(states,instructions):
            d = state
            d["instruction"]=instruction
            d['id']=int(idx.split('-')[1])
            d['seq']=seq
            parsed.append(d)
            seq+=1
df_scone = pd.DataFrame(parsed)
df_scone

,1,2,3,4,5,6,7,instruction,id,seq
0,_,g,p,o,g,r,y,INIT,1830,0
1,_,g,p,_,g,r,y,throw out the orange chemical,1830,1
2,_,_,p,_,g,r,yg,"then, add the leftmost beaker of green chemica...",1830,2
3,_,_,p,_,g,r,bb,mix it,1830,3
4,_,_,p,_,_,r,bbg,"then, add the remaining green chemical to it",1830,4
...,...,...,...,...,...,...,...,...,...,...
1465,r,_,gg,o,g,r,y,throw out second beaker,2128,1
1466,r,_,gg,oy,g,r,_,pour yellow beaker into orange one,2128,2
1467,_,_,gg,oyr,g,r,_,pour first beaker into fourth one,2128,3
1468,_,_,gg,bbb,g,r,_,it turns completely brown,2128,4


In [68]:
df_scone_first = df_scone.query("seq==0")
df_scone_first["sets"] = df_scone_first.apply(lambda r: [set(filter(lambda c:c in 'rgbyop', r[i])) for i in range(1,8)],axis=1)
scone_first=dict(df_scone_first[["id", "sets"]].values)

<ipython-input-68-3db1e4b23f61>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scone_first["sets"] = df_scone_first.apply(lambda r: [set(filter(lambda c:c in 'rgbyop', r[i])) for i in range(1,8)],axis=1)


In [77]:
df_scone_last = df_scone.query("seq==5")
df_scone_last["sets"] = df_scone_last.apply(lambda r: [set(filter(lambda c:c in 'rgbyop', r[i])) for i in range(1,8)],axis=1)
scone_last=dict(df_scone_last[["id", "sets"]].values)

<ipython-input-77-085eb1006f0a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scone_last["sets"] = df_scone_last.apply(lambda r: [set(filter(lambda c:c in 'rgbyop', r[i])) for i in range(1,8)],axis=1)


In [13]:
from instruction_parsing import program, Commands
import annotation_io
import read_data
annotation_io.set_recipe_source("scone")
from edit_distance import edit_distance


def read_annotation(annotation_id, mturk_batch=None):
    if mturk_batch:
        annotation = annotation_io.mturk_annotation(mturk_batch, annotation_id)
    else:
        annotation = annotation_io.get_annotation(annotation_id)
    return annotation

def annotator_agreement(annotation1,annotation2, ignore_commands=[]):
    lst1 = [(x.command.name,x.arg,x.resource) for x in program(annotation1) if x.command not in ignore_commands]
    lst2 = [(x.command.name,x.arg,x.resource) for x in program(annotation2) if x.command not in ignore_commands]
    dst,_ = edit_distance(lst1, lst2)
    return 1-dst/(len(lst1)+len(lst2))


In [14]:
ls(mturk_path)

../mturk/results/4335716.json
../mturk/results/Batch_4430799_batch_results.csv
../mturk/results/Batch_4635581_batch_results.csv
../mturk/results/results_approved.csv
../mturk/results/4289268.json
../mturk/results/Batch_4258572_batch_results.csv
../mturk/results/Batch_4580035_scone_results.csv
../mturk/results/Batch_4286064_batch_results.csv
../mturk/results/Batch_4438769_batch_results.csv
../mturk/results/Batch_4301520_batch_results.csv
../mturk/results/Batch_4289268_batch_results.csv
../mturk/results/Batch_4280306_batch_results.csv
../mturk/results/4286064.json
../mturk/results/4301520.json
../mturk/results/Batch_4335716_batch_results.csv


In [111]:
def load_results(batch):
    df = pd.read_csv(str(next(mturk_path.rglob(f"*{batch}*.csv"))))
    #return df
    #hack:
#     df = df.rename(columns={"Input.v11":"Answer.Title", "Input.v3": "Answer.ID"})
#     df = df[[c for c in df.columns if "Answer." in c or 'Id' in c]]
    df = df.rename(columns={c:c.split('.',1)[1] for c in df.columns if '.' in c})
#     df = df.drop("id", axis=1).rename(columns={"ID": "id"})
    return df.set_index("AssignmentId")
batch = 4635581 # 4580035 # 93
df=load_results(batch)
# df = df.loc[df["events"]!='[]',["HITId", "id", "events","feedback","msgbox_count","seconds_spent"]]
df["events"]=df["events"].apply(json.loads)
df

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,v12,v13,events,feedback,id,msgbox_count,seconds_spent,status,Approve,Reject
AssignmentId,,,,,,,,,,,,,,,,,,,,,
3FTOP5WARLJ2ORN5Q9G21TZ6WB10JT,3JMQI2OLG40QJHIQRHROKN4IXBRNDQ,394QQ8LGQ266852VV81MNHI782SZ3H,Beaker Instructions,Pour fluid from one beaker to an other accordi...,"laboratory,coding,recipe,program",$1.13,Tue Dec 21 04:26:47 PST 2021,5,BatchId:4635581;OriginalHitTemplateId:928390828;,660,...,"{""TOO"": ""Orange"", ""TPP"": ""Purple"", ""TYY"": ""Yel...",[],[],{},2065,0,[],1,NaN,NaN
3VP0C6EFSMRN3AF5VV9BQM1ED6S6MA,3JMQI2OLG40QJHIQRHROKN4IXBRNDQ,394QQ8LGQ266852VV81MNHI782SZ3H,Beaker Instructions,Pour fluid from one beaker to an other accordi...,"laboratory,coding,recipe,program",$1.13,Tue Dec 21 04:26:47 PST 2021,5,BatchId:4635581;OriginalHitTemplateId:928390828;,660,...,"{""TOO"": ""Orange"", ""TPP"": ""Purple"", ""TYY"": ""Yel...",[],"[[{'start': '2020-01-01T00:00:00', 'end': '202...",{},2065,2,"[61,20,58,9,15,54]",1,NaN,NaN
3HHRAGRYXE0YLW2F6IR020HLYGZO9K,3JMQI2OLG40QJHIQRHROKN4IXBRNDQ,394QQ8LGQ266852VV81MNHI782SZ3H,Beaker Instructions,Pour fluid from one beaker to an other accordi...,"laboratory,coding,recipe,program",$1.13,Tue Dec 21 04:26:47 PST 2021,5,BatchId:4635581;OriginalHitTemplateId:928390828;,660,...,"{""TOO"": ""Orange"", ""TPP"": ""Purple"", ""TYY"": ""Yel...",[],"[[{'start': '2020-01-01T00:00:00', 'end': '202...",{},2065,5,"[120,1,1,6,5,51]",1,NaN,NaN
3CP1TO84PZW1D2BYGUGMJ79GB71525,3JMQI2OLG40QJHIQRHROKN4IXBRNDQ,394QQ8LGQ266852VV81MNHI782SZ3H,Beaker Instructions,Pour fluid from one beaker to an other accordi...,"laboratory,coding,recipe,program",$1.13,Tue Dec 21 04:26:47 PST 2021,5,BatchId:4635581;OriginalHitTemplateId:928390828;,660,...,"{""TOO"": ""Orange"", ""TPP"": ""Purple"", ""TYY"": ""Yel...",[],[],{},2065,0,[],1,NaN,NaN
3FQ5JJ512RJ0K7LU90ICDKVWJSIKNR,3JMQI2OLG40QJHIQRHROKN4IXBRNDQ,394QQ8LGQ266852VV81MNHI782SZ3H,Beaker Instructions,Pour fluid from one beaker to an other accordi...,"laboratory,coding,recipe,program",$1.13,Tue Dec 21 04:26:47 PST 2021,5,BatchId:4635581;OriginalHitTemplateId:928390828;,660,...,"{""TOO"": ""Orange"", ""TPP"": ""Purple"", ""TYY"": ""Yel...",[],"[[{'start': '2020-01-01T00:00:00', 'end': '202...",{},2065,1,"[62,84,25,11,19,34]",1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3P1L2B7AD7KTMY3OVFUQ798GY3JOL6,368IUKXGBA3R1TFOX9W2I6ZLG466PE,394QQ8LGQ266852VV81MNHI782SZ3H,Beaker Instructions,Pour fluid from one beaker to an other accordi...,"laboratory,coding,recipe,program",$1.13,Tue Dec 21 04:27:10 PST 2021,4,BatchId:4635581;OriginalHitTemplateId:928390828;,660,...,"{""TOO"": ""Orange"", ""TPP"": ""Purple"", ""TYY"": ""Yel...",[],"[[{'start': '2020-01-01T00:00:00', 'end': '202...",{},2100,3,"[37,39,42,22,50,36]",1,NaN,NaN
379J5II41UB7A7QZ11RU2XE2U1REL6,368IUKXGBA3R1TFOX9W2I6ZLG466PE,394QQ8LGQ266852VV81MNHI782SZ3H,Beaker Instructions,Pour fluid from one beaker to an other accordi...,"laboratory,coding,recipe,program",$1.13,Tue Dec 21 04:27:10 PST 2021,4,BatchId:4635581;OriginalHitTemplateId:928390828;,660,...,"{""TOO"": ""Orange"", ""TPP"": ""Purple"", ""TYY"": ""Yel...",[],[],{},2100,0,[],0,NaN,NaN
3U4J9857OK6AOJPDNGOUIKQGWJ97BO,3UUIU9GZDA0TAH8H97VHVAMI9CFT5P,394QQ8LGQ266852VV81MNHI782SZ3H,Beaker Instructions,Pour fluid from one beaker to an other accordi...,"laboratory,coding,recipe,program",$1.13,Tue Dec 21 04:27:10 PST 2021,3,BatchId:4635581;OriginalHitTemplateId:928390828;,660,...,"{""TOO"": ""Orange"", ""TPP"": ""Purple"", ""TYY"": ""Yel...",[],"[[{'start': '2020-01-01T00:00:00', 'end': '202...",{},1837,4,"[85,54,35,131,21,46]",1,NaN,NaN


In [112]:
def nth_state(step_num):
    def nth_state_ext(j):
        try:
            first = collections.Counter([(i["resource"],i["text"]) for i in j[step_num] if not i["resource"].startswith("VALID")])
        except IndexError:
            return []
        arr = [set() for _ in range(7)]
        cmap = {"Brown": 'b', "Yellow": 'y', "Purple": 'p', "Red": 'r', "Green": 'g', "Orange": 'o'}
        nmap = ["ONE", "TWO", "THREE", "FOUR", "FIVE", "SIX", "SEVEN"]
        for i,b in enumerate(nmap):
            for (beaker,color),cnt in first.items():
                if beaker==b:
                    arr[i].add(cmap[color])
        return arr
    return nth_state_ext

df["first_state"]=df["events"].apply(nth_state(0))
df["last_state"]=df["events"].apply(nth_state(-1))

df["max_steps"]=df["events"].apply(max_steps)
df["first_scone"]=df["id"].map(scone_first)
df["last_scone"]=df["id"].map(scone_last)
df["first_ok"]=df["first_scone"]==df["first_state"]
df["last_ok"]=df["last_scone"]==df["last_state"]
df["Reject"]=df.apply(lambda row: "" if row["first_ok"] and any(row["events"]) else "X", axis=1)
df["Accept"]=df.apply(lambda row: "X" if row["first_ok"] and row["last_ok"] else "", axis=1)


In [113]:
df["first_ok"].value_counts()

True     729
False    408
Name: first_ok, dtype: int64

In [114]:
df["last_ok"].value_counts()

False    1055
True       82
Name: last_ok, dtype: int64

In [115]:
(df["last_ok"]&df["first_ok"]).value_counts()

False    1055
True       82
dtype: int64

In [116]:
df["Accept"].value_counts()

     1055
X      82
Name: Accept, dtype: int64

In [117]:
df.to_csv(mturk_path / "scone_state_test.csv")

# 

In [10]:
html=""
for assignment, row in df.iterrows():
    approved = assignment in df.index
    color = "green" if approved else "red"
    worker=row["WorkerId"]
    title=row["Title"]
    idx = row["id"]
    status = status_map[str(row["status"])]
    display_id=worker#assignment
    html+=f'(worker={worker}) <font color="{color}">{assignment}</font> <a href="http://127.0.0.1:8080/annotate/{batch}/{assignment}" target="_new">({idx}) {title} - {status}</a> '
    if int(row["status"])<0:
        html+=row["feedback"]
    html+='<br />'
display_html(html)

# Measure agreement

In [11]:
def annotator_agreement(lst1,lst2, ignore_commands=[],config=False):
    if not config:
        lst1 = [(x[0],x[1],x[2].rsplit('_',1)[0]) for x in lst1]
        lst2 = [(x[0],x[1],x[2].rsplit('_',1)[0]) for x in lst2]
    lst1 = [x for x in lst1 if x[0] not in ignore_commands]
    lst2 = [x for x in lst2 if x[0] not in ignore_commands]
    dst,_ = edit_distance(lst1, lst2)
    return 1-dst/(len(lst1)+len(lst2))

In [28]:
ignore_commands={"CHEF_CHECK","USE","STOP_USING"}
def couple_agreement(lst):
    ret=[]
    for l1, l2 in itertools.combinations(lst,2):
        ret.append(annotator_agreement(l1,l2, ignore_commands))
    return ret
df_agreement=df[df["n"]>0].groupby("id")["program"].apply(list).apply(couple_agreement)
df_agreement

KeyError: 'n'

In [29]:
df_agreement.apply(np.mean)

NameError: name 'df_agreement' is not defined

In [56]:
df_agreement.apply(np.mean).mean()

0.6394030325483624

# Measure time

In [61]:
df["total_seconds"]=df["seconds_spent"].apply(json.loads).apply(sum)
df[df["n"]>0].groupby("id")["total_seconds"].mean().mean()

519.362007168459